In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
from pandas.errors import EmptyDataError

PROJECT_ROOT = Path("..").resolve()
CORE8_DIR = PROJECT_ROOT / "artifact" / "core8"

STATE_TRACE_PATH = CORE8_DIR / "core8_03_refusal_state_trace_counterfactual.csv"
EVENT_LOG_PATH  = CORE8_DIR / "core8_03_refusal_event_log_counterfactual.csv"

assert STATE_TRACE_PATH.exists(), "state trace not found"
assert EVENT_LOG_PATH.exists(), "event log not found"

state_df = pd.read_csv(STATE_TRACE_PATH)

# --- event log는 비어 있을 수 있음 ---
try:
    event_df = pd.read_csv(EVENT_LOG_PATH)
    event_log_empty = False
except EmptyDataError:
    event_df = pd.DataFrame()
    event_log_empty = True

print("[state_df rows]:", len(state_df))
print("[event_df rows]:", len(event_df))
print("[event_log_empty]:", event_log_empty)

print("\n[state_df columns]")
print(state_df.columns.tolist())

print("\n[event_df columns]")
print(event_df.columns.tolist() if not event_log_empty else "EMPTY (no stage transitions)")

[state_df rows]: 180
[event_df rows]: 0
[event_log_empty]: True

[state_df columns]
['run_id', 'case_id', 'antibody_id', 'step', 'refusal_stage', 'refusal_mode', 'blocked_rate_window', 'veto_streak', 'action_toggle_rate', 'SoMS_cumsum_window', 'refusal_triggered', 'refusal_reason_code']

[event_df columns]
EMPTY (no stage transitions)


In [4]:
def summarize_refusal_stage(sub_df: pd.DataFrame) -> pd.Series:
    return pd.Series({
        "max_refusal_stage": sub_df["refusal_stage"].max(),
        "first_stage1_step": sub_df.loc[sub_df["refusal_stage"] >= 1, "step"].min(),
        "first_stage2_step": sub_df.loc[sub_df["refusal_stage"] >= 2, "step"].min(),
        "first_refusal_step": sub_df.loc[sub_df["refusal_stage"] >= 3, "step"].min(),
    })

table_stage = (
    state_df
    .groupby(["antibody_id", "case_id"])
    .apply(summarize_refusal_stage)
    .reset_index()
)

table_stage # Refusal Stage 도달 요약 (Case A vs B)

/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_34794/2696323385.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_refusal_stage)


,antibody_id,case_id,max_refusal_stage,first_stage1_step,first_stage2_step,first_refusal_step
0,antibody_A,A_ALWAYS_ALLOW,0.0,NaN,NaN,NaN
1,antibody_A,B_GOVERNED,0.0,NaN,NaN,NaN
2,antibody_B,A_ALWAYS_ALLOW,0.0,NaN,NaN,NaN
3,antibody_B,B_GOVERNED,0.0,NaN,NaN,NaN
4,antibody_C,A_ALWAYS_ALLOW,0.0,NaN,NaN,NaN
5,antibody_C,B_GOVERNED,0.0,NaN,NaN,NaN


In [5]:
def summarize_accumulative_metrics(sub_df: pd.DataFrame) -> pd.Series:
    return pd.Series({
        "blocked_rate_window_peak": sub_df["blocked_rate_window"].max(),
        "veto_streak_max": sub_df["veto_streak"].max(),
        "oscillation_peak": sub_df["action_toggle_rate"].max(),
        "SoMS_cumsum_window_peak": sub_df["SoMS_cumsum_window"].max(),
    })

table_acc = (
    state_df
    .groupby(["antibody_id", "case_id"])
    .apply(summarize_accumulative_metrics)
    .reset_index()
)

table_acc # Table 2 — 누적 기반 발동 “증거 수치” 요약

/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_34794/84437118.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_accumulative_metrics)


,antibody_id,case_id,blocked_rate_window_peak,veto_streak_max,oscillation_peak,SoMS_cumsum_window_peak
0,antibody_A,A_ALWAYS_ALLOW,0.00,0.0,0.000000,19.15
1,antibody_A,B_GOVERNED,0.10,1.0,0.222222,24.20
2,antibody_B,A_ALWAYS_ALLOW,0.00,0.0,0.000000,24.15
3,antibody_B,B_GOVERNED,0.10,1.0,0.222222,18.75
4,antibody_C,A_ALWAYS_ALLOW,0.00,0.0,0.000000,14.05
5,antibody_C,B_GOVERNED,0.25,1.0,0.500000,17.50


In [6]:
def pick_stage_row(stage: int):
    rows = state_df[state_df["refusal_stage"] == stage]
    if rows.empty:
        return None
    return rows.iloc[0]

sample_stage1 = pick_stage_row(1)
sample_stage2 = pick_stage_row(2)
sample_stage3 = pick_stage_row(3)

sample_stage1, sample_stage2, sample_stage3 # Table 3 — 정책 실행 로그 샘플 (있을 경우)

(None, None, None)

In [7]:
pivot_stage = table_stage.pivot(
    index="antibody_id",
    columns="case_id",
    values="max_refusal_stage"
)

pivot_acc = table_acc.pivot(
    index="antibody_id",
    columns="case_id",
    values="SoMS_cumsum_window_peak"
)

pivot_stage, pivot_acc # Case A vs Case B 비교 Pivot

(case_id      A_ALWAYS_ALLOW  B_GOVERNED
 antibody_id                            
 antibody_A              0.0         0.0
 antibody_B              0.0         0.0
 antibody_C              0.0         0.0,
 case_id      A_ALWAYS_ALLOW  B_GOVERNED
 antibody_id                            
 antibody_A            19.15       24.20
 antibody_B            24.15       18.75
 antibody_C            14.05       17.50)

In [8]:
def generate_policy_summary(row):
    if row["max_refusal_stage"] == 0:
        return (
            "No refusal stage was triggered. "
            "All fallback conditions were monitored but did not meet "
            "the accumulative thresholds required for escalation."
        )
    else:
        return (
            f"Refusal stage {int(row['max_refusal_stage'])} was reached, "
            "indicating execution of predefined fallback policy."
        )

table_stage["policy_summary"] = table_stage.apply(generate_policy_summary, axis=1)
table_stage[["antibody_id", "case_id", "policy_summary"]] # 정책 실행 요약 문장 자동 생성

,antibody_id,case_id,policy_summary
0,antibody_A,A_ALWAYS_ALLOW,No refusal stage was triggered. All fallback c...
1,antibody_A,B_GOVERNED,No refusal stage was triggered. All fallback c...
2,antibody_B,A_ALWAYS_ALLOW,No refusal stage was triggered. All fallback c...
3,antibody_B,B_GOVERNED,No refusal stage was triggered. All fallback c...
4,antibody_C,A_ALWAYS_ALLOW,No refusal stage was triggered. All fallback c...
5,antibody_C,B_GOVERNED,No refusal stage was triggered. All fallback c...


In [ ]:
print(
    "Core 8은 정책이 부재해서 거부를 발동하지 않은 것이 아니라, "
    "누적 기반 거버넌스 조건이 의도적으로 설계되었으며 "
    "그 조건이 충족되지 않았기 때문에 거부를 발동하지 않았다."
)